### 1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-pinecone langchainhub

/Users/lhj/inflearn-langchain-rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


### 2. Knowledge Base 구성을 위한 데이터 생성
- split 된 데이터 chunk를 LLM에게 전달하면 토큰 절약 가능 -> 비용 감소와 답변 생성시간 감소의 효과
- LangChain에서 다양한 TextSplitter들을 제공
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
    - chunk_size 는 split 된 chunk의 최대 크기
    - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('../tax_document/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# Upstage 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [21]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

### 3. 관련 정보를 가져오는 Retrieval (검색)
- Chroma에 저장한 데이터에서 질문 쿼리와 관련있는 데이터를 가져오는 과정
- similarity_search(): 유사도 검색 제공 
- `k` 값을 조절해서 얼마나 많은 연관 데이터를 불러올지 결정

In [23]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)

In [24]:
retrieved_docs

[Document(id='b6713663-3cf4-421d-bde0-6646f49e9b75', metadata={'source': './tax.docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015.

### 4. Augmentation(증강)을 위한 Prompt 활용
- Retrieval된 데이터를 프롬프트를 활용하여 LLM에 전달 
- LangChain에서 제공하는 프롬포트("rlm/rag-prompt")를 사용해도 되고, 본인이 직접 작성해도 된다. 

In [25]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [29]:
from langchain_core.prompts import ChatPromptTemplate

context = "\n\n---".join([doc.page_content for doc in retrieved_docs])

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 최고의 한국 소득세 전문가입니다. 
                주어진 context를 기반으로 질문에 답변하세요.
                - context에 답이 없으면 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답하세요
                - 최대 3-4문장으로 간결하게 답변하세요
                - 추측하지 말고 문서 내용만을 기반으로 답변하세요"""
    ),
    ("human", """Context: {context}
                Question: {question}"""
    )
])

### 5. 답변 생성 (Generation)
- 프롬포트를 LLM에게 전달하여 답변 생성 

In [27]:
# 7. LLM 호출
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

In [30]:
chat_value = rag_prompt.invoke({"context": context, "question": query})
ai_response = llm.invoke(chat_value)

print(ai_response.content)

제공된 문서에서는 연봉 5천만원인 직장인의 소득세 계산 방법에 대한 구체적인 정보가 없습니다. 문서에는 근로소득공제(제47조), 근로소득세액공제(제59조), 자녀세액공제(제59조의2) 등 관련 조항이 있으나, 특정 금액을 기준으로 한 소득세 산출 공식은 명시되어 있지 않습니다. 따라서 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답변드립니다. 

추가적으로 근로소득공제(5천만원 총급여 시 1,500만원 + (5천만원-1,500만원)×15% = 2,475만원)와 근로소득세액공제(총급여액 3,300만원 초과 시 계산식 적용 가능)는 확인할 수 있으나, 종합소득산출세액 계산에는 추가 정보(과세표준, 세율, 기타 공제 등)가 필요합니다.
